In [1]:
import os
os.getcwd()

'c:\\Users\\azaan\\OneDrive\\Documents\\GitHub\\cs410_LLM_project\\scripts'

In [2]:
# # import importlib.util
# # print(importlib.util.find_spec("safetensors") is not None)

# %pip install --upgrade pip
# # !pip install protobuf
# %pip install torch
# %pip install datasets
# %pip install accelerate 
# %pip install huggingface-hub 
# %pip install safetensors 
# %pip install tokenizers
# %pip install nltk
# %pip install scikit-learn

# # need to use % instead of ! for some reason
# %pip install transformers

# # !pip list
# # !pip show transformers

In [3]:
# import the dataset
import pandas as pd
import numpy as np
import os

# Directory path
MAIN_DIR = "C:\\Users\\azaan\\OneDrive\\Documents\\GitHub\\cs410_LLM_project\\data\\all_lectures.csv"

# Initialize an empty DataFrame

df = pd.DataFrame(columns=['Week Number', 'Lesson Number', 'Lesson Title', 'Transcript'])
# Read in csv to dataframe
df = pd.read_csv(MAIN_DIR)

# Display the resulting DataFrame
df.head()

,Week Number,Lesson Number,Lesson Title,Transcript
0,1,1,Natural Language Content Analysis,This lecture is about Natural Language of Cont...
1,1,2,Text Access,"In this lecture,\nwe're going to talk about th..."
2,1,3,Text Retrieval Problem,This lecture is about\nthe text retrieval prob...
3,1,4,Overview of Text Retrieval Methods,This lecture is a overview of\ntext retrieval ...
4,1,5,Vector Space Model - Basic Idea,This lecture is about the\nvector space retrie...


In [4]:
# clean up words in dataset -- this includes removing stopwords
import regex as re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, words, brown

nltk.download("stopwords")
nltk.download("words")
nltk.download("brown")
nltk.download("punkt")

lemmer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# initialize dictionary
global_dictionary  = set(words.words()) | set(brown.words())
global_dictionary = {word.lower() for word in global_dictionary}
remove_words = list(stop_words) # might need to use word_tokenize
remove_words.extend(['Play', 'video', 'starting', 'at', '::', 'follow', 'transcript', 'natural', 'language', 'lecture', 'processing']) # remove the common words that are included in transcript

# Now start actually cleaning the text
def clean_text(text):
    text = text.lower() # lowercase
    text = text.replace('\n', ' ') # remove newline indicator
    text = re.sub(r'[^a-zA-Z\s]', '', text) # case
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'http\S+|www\S+', '', text) # website
    text = re.sub(r'(\b\w+\b)(?: \1)+', r'\1', text) # remove duplicate next word after space
    text = re.sub(r'\b(?![aI]\b)\w\b', '', text)

    return text

# Remove stopwords and only keep words in dictionary
def remove_terms(text):
    text = clean_text(text)
    words = text.split()
    # filtered_words = [word for word in words if word not in remove_words] # remove stopwords
    filtered_words = [word for word in words if word in global_dictionary] # remove if not in global dictionary
    return " ".join(filtered_words)

# Tokenize reviews + remove stop words + filter only nouns
def tokenize_and_filter(text):
    words = nltk.word_tokenize(text)
    words = [word.lower() for word in words] # if word.lower() not in stop_words and len(word) > 2]
    # print(words)
    # pos_comment = nltk.pos_tag(words)
    # filtered = [word[0] for word in pos_comment if word[1] in ['NN']]
    return words #filtered

def lower_text(text):
    words = text.lower()
    return words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\azaan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\azaan\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\azaan\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\azaan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
pd.options.display.max_rows = 500

df['Transcript_Cleaned'] = df['Transcript'].apply(remove_terms)
df['Transcript_Cleaned'] = df['Transcript_Cleaned'].apply(lower_text)
# Skipping this in order to tokenize later
# df['Transcript_Cleaned'] = df['Transcript_Cleaned'].map(tokenize_and_filter)
df['Transcript_Cleaned']

0     this lecture is about natural language of cont...
1     in this lecture were going to talk about the t...
2     this lecture is about the text retrieval probl...
3     this lecture is a overview of text retrieval m...
4     this lecture is about the vector space retriev...
5     in this lecture were going to talk about how t...
6     in this lecture we are going to talk about how...
7     in this lecture we continue the discussion of ...
8     this lecture is about document length normaliz...
9     this lecture is about the implementation of te...
10    this lecture is about the inverted index const...
11    this lecture is about how to do faster search ...
12    this lecture is about evaluation of text retri...
13    this lecture is about the basic measures for e...
14    this lecture is about how we can evaluate a ra...
15    so average precision is computer for just one ...
16    this lecture is about how to evaluate the text...
17    this lecture is about some practical issue

In [6]:
import random

def extract_window_labels(df, window_size, num_samples):
  """
  Extracts window/label pairs from a DataFrame with batching, skip logic, and error checks.

  Args:
    df: A DataFrame containing text sections in the "Transcript_Cleaned" column.
    window_size: The size of the window for capturing context.
    num_samples: The number of window/label pairs to generate per batch.

  Returns:
    A list of dictionaries containing window and label pairs.
  """
  new_data = []

  for i in range(len(df)):
    text = df.loc[i, "Transcript_Cleaned"]
    remaining_words = len(text.split())
    processed_words = 0

    # Process batches of 100 words
    while remaining_words >= 100:
      # Skip processing the last 50 words in subsequent batches
      if remaining_words < 100:
        break

      # Check if sufficient words remain for another batch
      if remaining_words < window_size + 1:
        break

      batch_text = text[processed_words:processed_words + 100]
      batch_words = batch_text.split()

      # Generate samples within the batch
      for _ in range(num_samples):
        # Check for valid sampling range within the batch
        max_start_idx = len(batch_words) - window_size - 1
        start_idx = random.randint(window_size, max_start_idx)
        end_idx = start_idx + window_size

        # Create window and label strings
        window_text = " ".join(batch_words[start_idx:end_idx])
        label = batch_words[end_idx]

        # Append pair to new data
        new_data.append({"window": window_text, "label": label})

      processed_words += 100
      remaining_words -= 100

  return new_data

new_df = extract_window_labels(df, 5, 15)

In [7]:
new_df = pd.DataFrame.from_dict(new_df)
new_df = new_df.drop_duplicates()
new_df

,window,label
0,you see from this picture,this
1,content analysis as you see,from
2,from this picture this is,real
3,language of content analysis as,you
4,of content analysis as you,see
...,...,...
23658,meaningful for other sequences of,data
23659,might be also meaningful for,other
23660,for other sequences of data,we
23663,data we also talked a,lo


In [8]:
# now we have our dataframe, let's use it!
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import json
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report

c:\Users\azaan\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# lets save the label values as integers, and save the indexes for them as well
results = set(new_df['label'])

# save a vocabulary of labels to predict with number indexes
word_to_index = {word: idx for idx, word in enumerate(results)}
index_to_word = {idx: word for word, idx in word_to_index.items()}

# convert labels into indices
def words_to_indices(words):
    return [word_to_index[word] for word in words]
new_df['label_indices'] = words_to_indices(new_df['label'])

# create function to reverse this, for output
def indices_to_words(indices):
    return [index_to_word[idx] for idx in indices]


In [10]:
new_df
df

,Week Number,Lesson Number,Lesson Title,Transcript,Transcript_Cleaned
0,1,1,Natural Language Content Analysis,This lecture is about Natural Language of Cont...,this lecture is about natural language of cont...
1,1,2,Text Access,"In this lecture,\nwe're going to talk about th...",in this lecture were going to talk about the t...
2,1,3,Text Retrieval Problem,This lecture is about\nthe text retrieval prob...,this lecture is about the text retrieval probl...
3,1,4,Overview of Text Retrieval Methods,This lecture is a overview of\ntext retrieval ...,this lecture is a overview of text retrieval m...
4,1,5,Vector Space Model - Basic Idea,This lecture is about the\nvector space retrie...,this lecture is about the vector space retriev...
5,1,6,Vector Space Retrieval Model - Simplest Instan...,In this lecture we're going to talk about how ...,in this lecture were going to talk about how t...
6,2,1,Vector Space Model - Improved Instantiation,"In this lecture, we are going to talk about ho...",in this lecture we are going to talk about how...
7,2,2,TF Transformation,"In this lecture, we continue\nthe discussion o...",in this lecture we continue the discussion of ...
8,2,3,Doc Length Normalization,This lecture is about Document Length Normaliz...,this lecture is about document length normaliz...
9,2,4,Implementation of TR Systems,This lecture is about the implementation\nof t...,this lecture is about the implementation of te...


In [11]:
# might just use the normal df for now

import re
import json
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.15)

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))


Train dataset length: 81
Test dataset length: 15


In [12]:
# # Display some samples
# for example in train_dataset:
#     print(len(tokenizer(example["Transcript_Cleaned"]).input_ids))


In [13]:
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
import datasets
from datasets import Dataset, DatasetDict

tokenizer = AutoTokenizer.from_pretrained("gpt2-large")
tokenizer.pad_token = tokenizer.eos_token
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [14]:
max_seq_length = 1024

def tokenization(example):
    return tokenizer(example["Transcript_Cleaned"], padding='max_length', truncation=True, max_length=max_seq_length, return_tensors="pt")

train_dataset = train_dataset.map(tokenization)
test_dataset = test_dataset.map(tokenization)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Map: 100%|██████████| 15/15 [00:00<00:00, 219.77 examples/s]


In [15]:
# Do not run local, only run with GPU on Colab

# from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

# model = AutoModelWithLMHead.from_pretrained("gpt2-medium").to(device)

# training_args = TrainingArguments(
#         output_dir='/content/drive/MyDrive/fine-tuned-model',
#         overwrite_output_dir=True,
#         num_train_epochs=3,
#         per_device_train_batch_size=2, # batch size for training
#         # per_device_eval_batch_size=64,  # batch size for evaluation
#         # eval_steps=100, # Number of update steps between two evaluations.
#         save_steps=3, # after # steps model is saved
#         # warmup_steps=500,# number of warmup steps for learning rate scheduler
#         gradient_accumulation_steps=4, # or any larger value
#         gradient_checkpointing=True # SUPER IMPORTANT!!! MAKES IT SO DOESN'T TAKE UP TOO MUCH RAM AND ISNTEAD TRANSFERS TO DISK!!!
#     )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     tokenizer=tokenizer
# )

# trainer.train()

# save model to output directory from training arguments
# trainer.save_model()

In [16]:
# Reloading the model back in
from transformers import AutoModelWithLMHead, AutoConfig

model_path = 'C:\\Users\\azaan\\OneDrive\\Documents\\GitHub\\cs410_LLM_project\\fine-tuned-model\\checkpoint-15'
config = AutoConfig.from_pretrained(model_path)
loaded_model = AutoModelWithLMHead.from_pretrained(model_path, config=config, use_safetensors=True).to(device)

c:\Users\azaan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\auto\modeling_auto.py:1564: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [17]:
# Generate text from a prompt
prompt = "In the lecture regarding an overview of text retreival methods, how did we define the ranking function"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text using the trained model
output = loaded_model.generate(input_ids, max_length=250, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)

c:\Users\azaan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\azaan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [20]:
# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

In the lecture regarding an overview of text retreival methods, how did we define the ranking function?

We defined the rank function as follows:
 and the function is defined as: where is the number of words in the text and is a function that takes a word and returns the word in a particular order.
 the first term is simply the weight of the words that are in this particular word order and this is just the sum of all the weights of these words. the second term here is called the average weight and it is basically the weighted sum over all of those words and we can see that this sum is equal to the total weight for all words of this word. so this means that the overall weight is zero and that is because the order of each word is not important here and so the final term in our ranking is also zero because we have zero weight on the last word so we are not going to rank the whole text in any particular way and in fact we will rank all text based on this weight so that we get the best rank

In [ ]:
# come up with like 5 good prompts from the lectures to ask the model
# Question 1: When it comes to mutual information, does "meats" have more mutual information with "eats" or "the"?
# Question 2: Can you explain PageRank to me, and let me know how it works in detail?
# Question 3: How does Logistic Ordinal Regression help in text classification?
# Question 4: How can I prepare for exam 1?

In [25]:
# use flask in order to generate answer to question

def answer(question):
    prompt = question

    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generate text using the trained model
    output = loaded_model.generate(input_ids, max_length=350, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return generated_text

In [27]:
from flask import Flask, render_template, request


app = Flask(__name__)

#This code creates the initial website
@app.route("/")
def html():
    return render_template("index.html", question = "", answer="")

#This code will update the website when the user submits a question
@app.route("/update", methods=["GET","POST"])
def update():
    question = request.form['input']
    response = ''
    if len(question) > 250:
        question = 'Question: ' + question
        response = 'Your question is to long for our model, please make it shorter than 50 characters. Thank You!'
    elif len(question) is 0:
        question = 'Please Enter a Question'
    else:
        question = 'Question: ' + question
        response = 'Answer: ' +  answer(question) 
    return render_template("index.html", question=question, answer=response)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


<>:19: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:19: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\azaan\AppData\Local\Temp\ipykernel_15664\2492220966.py:19: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif len(question) is 0:
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Dec/2023 00:57:01] "GET / HTTP/1.1" 200 -
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
127.0.0.1 - - [22/Dec/2023 00:58:39] "POST /update HTTP/1.1" 200 -
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
127.0.0.1 - - [22/Dec/2023 00:59:46] "PO